In [4]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import pywt
from natsort import natsorted

In [32]:
IMG_PATH = '../data/senales_transito/'

In [17]:
def get_vector_from_image(image, iterations):
    """
    get_vector_from_image obtiene el vector característico de la imagen image

    :param image: Imagen en formato vector.
    :param iterations: Entero que indica la cantidad de veces que se aplica el wavelet a la imagen.
    :return LL.flatten(): Vector característico en 1D.
    """
    LL, (LH, HL, HH) = pywt.dwt2(image, 'haar')
    for _ in range(iterations - 1):
        LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
    return LL.flatten()


In [39]:
def generate_data(src_dir, iterations, width=128, height=128):

    data = []
    file_names = []
    fv = []
    for class_dir in natsorted(os.listdir(src_dir)):
        for train_img in natsorted(os.listdir(src_dir + class_dir)):
            image_path = f"{src_dir}{class_dir}/{train_img}"
            file_names.append(train_img)
            img = Image.open(image_path)
            fv = get_vector_from_image(img, iterations)
            data.append(fv)

    df = pd.DataFrame(data, index = file_names)
    print(f"Tamaño del vector caracteristica con {iterations} cortes: {len(fv)}")
    return df


In [40]:
for i in range(3,8):
    data = generate_data(src_dir= IMG_PATH, iterations=i, width=512, height=512)
    data = (data-data.min())/(data.max()-data.min())

    for pos in range(len(data)):
        data.iloc[pos].to_csv(f"data/senales_transito/{i}_cortes/{data.index[pos][:-4]}.csv",index=False, index_label=None, header=False)

Tamaño del vector caracteristica con 3 cortes: 480
Tamaño del vector caracteristica con 4 cortes: 240
Tamaño del vector caracteristica con 5 cortes: 120
Tamaño del vector caracteristica con 6 cortes: 60
Tamaño del vector caracteristica con 7 cortes: 60


In [41]:
    data = []
    
    for train_img in natsorted(os.listdir('data/senales_transito/3_cortes/')):
        data.append(train_img)
    np.savetxt("data/senales_transito/paths.csv", data, delimiter=',', fmt="%s")
